<a href="https://colab.research.google.com/github/deb-kit2/coHereHack/blob/main/embedding4data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install cohere
! pip install -U sentence_transformers
# for cosine similarity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Imports

In [2]:
import os
import copy
import random

import cohere
from sentence_transformers import util

# Preparing Data

In [3]:
path = "data/"

# asset
dataorig = []
datasimple = []

fp = open(path + "assetValid.txt", "r", encoding = "utf-8")
for line in fp :
    dataorig.append(line)
fp.close()

temp = copy.deepcopy(dataorig)
for i in range(9) :
    dataorig.extend(temp)

for i in range(10) :
    fp = open(path + "assetValid_" + str(i) + ".txt", "r", encoding = "utf-8")
    for line in fp :
        datasimple.append(line)
    fp.close()

# medText
normal = []
simple = []

fp = open(path + "meddecxt_data.txt", "r", encoding = "utf-8")
for line in fp :
    t = line.split("--separator--")
    
    for tc in t :
        x = tc.split("--EOI--")

        y = x[1].strip()
        x = x[0].split("--simplificataion--")[-1].strip()

        normal.append(x)
        simple.append(y)

fp.close()

dataorig.extend(normal)
datasimple.extend(simple)

## ls
lsOrig = []
lsSimple = []

fp = open(path + "ls_original.txt", "r", encoding = "utf-8")
for line in fp :
    lsOrig.append(line)
fp.close()

fp = open(path + "ls_simplified.txt", "r", encoding = "utf-8")
for line in fp :
    lsSimple.append(line)
fp.close()

dataorig.extend(lsOrig)
datasimple.extend(lsSimple)

## ss
ssOrig = []
ssSimple = []

fp = open(path + "ss_original.txt", "r", encoding = "utf-8")
for line in fp :
    ssOrig.append(line)
fp.close()

temp = copy.deepcopy(ssOrig)
ssOrig.extend(temp)

fp = open(path + "ss_ls_simplified.txt", "r", encoding = "utf-8")
for line in fp :
    ssSimple.append(line)
fp.close()

fp = open(path + "ss_simplified.txt", "r", encoding = "utf-8")
for line in fp :
    ssSimple.append(line)
fp.close()

dataorig.extend(ssOrig)
datasimple.extend(ssSimple)

# shuffle and merge
random.Random(42).shuffle(dataorig)
random.Random(42).shuffle(datasimple)

In [4]:
print(len(dataorig), len(datasimple))

28890 28890


## Generating Embeddings

In [11]:
co = cohere.Client('bCLcz9DdEUqlFHWflXJG25sofkawR3PraHbBsOKY')

response = co.embed(model = 'small', texts = dataorig)
dataOrig_embeddings = response.embeddings

response = co.embed(model = 'small', texts = datasimple)
dataSimple_embeddings = response.embeddings

## Filtering data, based on similarity with the reference text

In [12]:
count = 6921
fp = open("mergedAssetMedPA.txt", "w", encoding = "utf-8")

for i in range(len(dataOrig_embeddings)) :
    similarity = float(util.pytorch_cos_sim(dataOrig_embeddings[i], dataSimple_embeddings[i]))

    if similarity > 0.92 or similarity < 0.30 :
        continue

    count += 1
    text = "Simplify the below text:\n" + "Complex input: " + dataorig[i].strip() + "\nSimplified output: " + datasimple[i].strip() + "\n--separator--\n"
    fp.write(text)

fp.close()

print(count)

6921
